In [ ]:
import sys
import os
from importlib import reload

# path to cluster_roi_master module
cluster_roi_master_folder = r"C:\github\spatially_constrained_spectral_clustering\cluster_roi_master"
# path to this repository
segmentation_folder = r"C:\github\spatially_constrained_spectral_clustering"
# add paths
sys.path.append(cluster_roi_master_folder)
sys.path.append(segmentation_folder)
# import and reload figure_utils module
import figure_utils

# import specific functions from the cluster_roi_master module
from make_local_connectivity_scorr import *
from make_local_connectivity_tcorr import *


In [ ]:
# Step 1. Calculate hierarchical clustering at the individual level

def pyClusterROI_corr(project_dict, sub_N, run_N, mask_name, corr_type, threshold, atlas_version='2mm'):
    # Calculate temporal/spatial correlation, uses the pyClusterROI toolbox from Cameron Craddock (https://github.com/ccraddock/cluster_roi)
    # project_dict: dictionary containing the project information
    # sub_N: subject number
    # run_N: run number
    # mask_name: name of the mask, the mask will be loaded from the associated atlas folder indicated in project_dict
    # corr_type: type of correlation ('t' for temporal or 's' for spatial)
    # threshold: threshold for the correlation matrix
    dataset = project_dict['Dataset']
    session = project_dict['Session']
    task = project_dict['Task']
    specie = project_dict['Specie']
    datafolder = project_dict['Datafolder']
    
    # input_folder has the normalized data
    input_folder = datafolder + os.sep + dataset + os.sep + 'normalized' + os.sep + specie + '-sub-' + str(sub_N).zfill(2)
    input_file = input_folder 
    input_file = input_file + os.sep + specie + '-sub-' + str(sub_N).zfill(2) + '_ses-' + session + '_task-' + task + '_run-' + str(run_N).zfill(2) + '.nii.gz'
    # output_folder has the results of the clustering
    output_folder = datafolder + os.sep + dataset + os.sep + 'hierarchical_clustering' + os.sep + specie + '-sub-' + str(sub_N).zfill(3)
    output_file = output_folder 
    output_file = output_file +  os.sep + specie + '-sub-' + str(sub_N).zfill(3) + '_task-' + task + '_run-' + str(run_N).zfill(2) + '-' + corr_type + 'corr.npy'
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    maskfile = os.path.join(datafolder, dataset, 'masks', f'{mask_name}.nii.gz')
    
    # run make_local_connectivity based on the type of function 
    if corr_type == 't':
        make_local_connectivity_tcorr(input_file, maskfile, output_file, threshold)
    elif corr_type == 's':
        make_local_connectivity_scorr(input_file, maskfile, output_file, threshold)
    else:
        print('Error: invalid correlation type, valid options are "t" for temporal or "s" for spatial')
    

# Change here to process a different dataset
selected_dataset = 'Knee'
project_dict,_ = figure_utils.get_project_dict(selected_dataset)
datafolder = r"C:\data"
# assign to project dict
project_dict['Datafolder'] = datafolder


run_N = 1
mask_name = 'b_GreyMatter2mm'
threshold = 0.5
corr_type = 't'

for sub_N in project_dict['Participants']:
    # run the function for each subject
    pyClusterROI_corr(project_dict, sub_N, run_N, mask_name, corr_type, threshold)


7709 # of non-zero voxels in the mask
Processing voxel # 0


c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Processing voxel # 1000
Processing voxel # 2000
Processing voxel # 3000
Processing voxel # 4000
Processing voxel # 5000
Processing voxel # 6000
Processing voxel # 7000
Finished processing C:\data\CAPS_Knee\normalized\D-sub-02\D-sub-02_ses-01_task-resting_state_run-01.nii.gz, total voxels processed: 7709
7709 # of non-zero voxels in the mask
Processing voxel # 0
Processing voxel # 1000
Processing voxel # 2000
Processing voxel # 3000
Processing voxel # 4000
Processing voxel # 5000
Processing voxel # 6000
Processing voxel # 7000
Finished processing C:\data\CAPS_Knee\normalized\D-sub-05\D-sub-05_ses-01_task-resting_state_run-01.nii.gz, total voxels processed: 7709
7709 # of non-zero voxels in the mask
Processing voxel # 0
Processing voxel # 1000
Processing voxel # 2000
Processing voxel # 3000
Processing voxel # 4000
Processing voxel # 5000
Processing voxel # 6000
Processing voxel # 7000
Finished processing C:\data\CAPS_Knee\normalized\D-sub-06\D-sub-06_ses-01_task-resting_state_run-01.nii.

In [ ]:
# Step 2. Hierarchical clustering group at the group level
from importlib import reload
import importlib
import nibabel as nb
import imp
import os
import connectivityR
reload(connectivityR)
import shutil
import numpy as np  # Ensure numpy is imported

reload(connectivityR)

def load_module(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    if spec is None:
        raise ImportError(f"Cannot find module {module_name} at {file_path}")
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

# Dynamically load additional modules

cluster_roi_master_path = r"C:\github\spatially_constrained_spectral_clustering\cluster_roi_master"
datafolder = r"C:\data"

group_mean_path = os.path.join(cluster_roi_master_path, 'group_mean_binfile_parcellation.py')
make_image_path = os.path.join(cluster_roi_master_path, 'make_image_from_bin_renum.py')

groupMean = load_module('group_mean_binfile_parcellation', group_mean_path)
make_img = load_module('make_image_from_bin_renum', make_image_path)

def group_mean(datafolder, subs_possible, runs_possible, corr_type, number_of_clusters, mask_name, dataset, specie, task='rest', rnd=False, rnd_range=[0,100], rnd_N_group=1):
    # specie = 'D'
    # The name of the maskfile that we will be using
    corr_conn_files = []    
    for sub_N in subs_possible:
        # print(f"sub_N: {sub_N}")
        for run_N in runs_possible:
            if rnd:  # if rnd is True, we need to add the rnd number to the file name
                input_folder = os.path.join(
                    datafolder, dataset, 'hierarchical_clustering', 'rnd',
                    f'{specie}-sub-{str(sub_N).zfill(3)}'
                )
                # sample within the rnd range
                rnd_N = np.random.randint(rnd_range[0], rnd_range[1]+1)
                # create the file name adding the rnd number
                input_file = f'{specie}-sub-{str(sub_N).zfill(3)}_{str(rnd_N).zfill(4)}'
            else:  # regular file name
                input_folder = os.path.join(
                    datafolder, dataset, 'hierarchical_clustering',
                    f'{specie}-sub-{str(sub_N).zfill(3)}'
                )
                input_file = f'{specie}-sub-{str(sub_N).zfill(3)}'
          
            # non rnd version
            input_file += f'_task-{task}_run-{str(run_N).zfill(2)}-{corr_type}corr.npy'

            full_input_path = os.path.join(input_folder, input_file)
            # print(f'added {full_input_path}')
            corr_conn_files.append(full_input_path)

    maskFile = os.path.join(datafolder, dataset, 'masks', f'{mask_name}.nii.gz')
    # Load maskFile
    mask = nb.load(maskFile)
    mask_data = mask.get_fdata()
    # Count the number of voxels in the mask
    n_voxels = np.count_nonzero(mask_data)
    if rnd:  # if rnd is True, we need to add the padded rnd_N_group number to the file name
        file_out = os.path.join(
            datafolder, dataset, 'hierarchical_clustering', 'rnd',
            f'{specie}-group-{corr_type}corr-{mask_name}_{str(rnd_N_group).zfill(4)}'
        )
        # generate a list of all files that should be generated
        list_of_files = [f'{file_out}_{str(K)}.npy' for K in number_of_clusters]
        # check if all files exist
        if all([os.path.exists(f) for f in list_of_files]):
            print(f'All files {file_out} already exist, skipping')
            return
        else:
            print(f'Processing {file_out}_{str(number_of_clusters[0])}.npy')
    else:
        file_out = os.path.join(
            datafolder, dataset, 'hierarchical_clustering',
            f'{specie}-group-{corr_type}corr-{mask_name}'
        )
        # print
        print(f'Processing {file_out}_{str(number_of_clusters[0])}.npy')

    # Call the group mean function
    groupMean.group_mean_binfile_parcellate(corr_conn_files, file_out, number_of_clusters, n_voxels)

    for K in number_of_clusters:
        if rnd:
            npyFile = os.path.join(
                datafolder, dataset, 'hierarchical_clustering', 'rnd',
                f'{specie}-group-{corr_type}corr-{mask_name}_{str(rnd_N_group).zfill(4)}_{K}'
            )
        else:
            npyFile = os.path.join(
                datafolder, dataset, 'hierarchical_clustering',
                f'{specie}-group-{corr_type}corr-{mask_name}_{K}'
            )
        # Generate NIfTI image from binary renumbered data
        make_img.make_image_from_bin_renum(f'{npyFile}.nii.gz', f'{npyFile}.npy', maskFile)

        # Load NIfTI file
        img = nb.load(f'{npyFile}.nii.gz')
        img_shape = img.shape
        img_data = img.get_fdata().flatten()

        # Check if folder matching npyFile exists, if it does, remove it
        if os.path.exists(npyFile):
            if os.path.isdir(npyFile):
                shutil.rmtree(npyFile)
            else:
                os.remove(npyFile)
        os.makedirs(npyFile)

        # Get unique clusters
        cluster_N = sorted(set(img_data[img_data > 0]))
        for cluster in cluster_N:
            cluster = int(cluster)
            # Folder for the cluster
            cluster_file = os.path.join(npyFile, f'cluster_{str(cluster).zfill(3)}.nii.gz')
            cluster_img = np.zeros_like(img_data)
            cluster_img[img_data == cluster] = 1

            cluster_img = cluster_img.reshape(img_shape)
            cluster_img_nifti = nb.Nifti1Image(cluster_img, img.affine)
            nb.save(cluster_img_nifti, cluster_file)


mask_name = 'b_GreyMatter2mm'
corr_type = 't'
specie = 'D'
dataset = 'CAPS_Knee'
task = 'resting_state'
if dataset == 'CAPS_K9':
    subs_possible = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,22,23,27,28,30,31,34,36,37]
elif dataset == 'CAPS_epi':
    subs_possible = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,22,23,27,28,30,31,34,36,37]
elif dataset == 'CAPS_Knee':
    subs_possible = [2,5,6,7,9,11,12,14,18,27]
else:
    # issue exception
    raise ValueError(f"Unknown dataset: {dataset}")

runs_possible = [1]
number_of_clusters = [20,40,60,80,100,120,140,160,180,200,220,240,260,280,300]

group_mean(datafolder,subs_possible,runs_possible,corr_type,number_of_clusters,mask_name,dataset, specie ,task)

Processing C:\data\CAPS_Knee\hierarchical_clustering\D-group-tcorr-b_GreyMatter2mm_20.npy
Started at 1760880132.8661323
Reading C:\data\CAPS_Knee\hierarchical_clustering\D-sub-002\D-sub-002_task-resting_state_run-01-tcorr.npy as a .npy file
data_w [1.         0.50435544 1.         ... 1.         0.64379326 0.78911662]
data_w shape (50066,)
indices_i [   0   33    1 ... 7708 7660 7669]
max indices_i 7708
indices_j [   0    0    1 ... 7708 7708 7708]
max indices_j 7708
n_voxels 7709
Reading C:\data\CAPS_Knee\hierarchical_clustering\D-sub-005\D-sub-005_task-resting_state_run-01-tcorr.npy as a .npy file
data_w [1.         1.         1.         ... 0.65737304 0.70815641 1.        ]
data_w shape (33874,)
indices_i [   0    1    2 ... 7705 7659 7708]
max indices_i 7708
indices_j [   0    1    2 ... 7708 7708 7708]
max indices_j 7708
n_voxels 7709
Adding 1
Reading C:\data\CAPS_Knee\hierarchical_clustering\D-sub-006\D-sub-006_task-resting_state_run-01-tcorr.npy as a .npy file
data_w [1.        

C:\github\CAPS\cluster_roi_master\python_ncut_lib.py:161: RuntimeWarning: SVD did not converge after maximum retries. Proceeding with the best result obtained.
  warnings.warn("SVD did not converge after maximum retries. Proceeding with the best result obtained.", RuntimeWarning)


Finished discretisation for k = 100 after 26.418443202972412 seconds

Finished clustering for k = 100 after 26.439993619918823 seconds

Finished discretisation for k = 120 after 35.72533822059631 seconds

Finished clustering for k = 120 after 35.752628326416016 seconds

Finished discretisation for k = 140 after 48.984681367874146 seconds

Finished clustering for k = 140 after 49.01491570472717 seconds

Finished discretisation for k = 160 after 64.02410340309143 seconds

Finished clustering for k = 160 after 64.05585169792175 seconds

Finished discretisation for k = 180 after 82.20767402648926 seconds

Finished clustering for k = 180 after 82.2444019317627 seconds

Finished discretisation for k = 200 after 103.30474901199341 seconds

Finished clustering for k = 200 after 103.36059951782227 seconds

Finished discretisation for k = 220 after 127.37049722671509 seconds

Finished clustering for k = 220 after 127.41813540458679 seconds

Finished discretisation for k = 240 after 153.720505475